# Data Processing

The client provided the data in an excel sheet - `data/main.xlsx`. There are five sheets (seawater, copper, cadmium, lead, and mix) but we didn't work with mix because we wanted to focus on building a simple model first. 

Each row of each sheet is a water sample and the output of the readings when the volts were passed through them. 

My first goal was to get everything in a format that is easier to work with and that I am more familiar with: csv files and pandas dataframes. 

Note that since I will be building an LSTM model in PyTorch, the inputs are expected to be of the shape: ???. But DataFrames are easiest to work with column wise. Each column represents a sequence and so I have added the class labels as an extra row. It was more convenient to access each sequence with column indexing e.g. `copper['seq_1']` than having the columns be voltage indexes (0, 1, 2, 3...). Moreover, in later notebooks, I will use only a subset of sequences to feed into the model and it felt more natural to select specific columns rather than rows of data to use. I will transpose the dataframes before feeding them into the torch model.

## The Process

1. Extract the sheets from the excel file into dataframes
2. Drop unnecessary columns
3. Create unique, descriptive column names for each sample (including metal, concentration and sample number)
4. Transfrom the dataframes from wide-form to long-form
5. Create a voltage column
6. Reset index to be unique

It's all in the `data.py` file - rename it to data_processing or something. 

In [6]:
import numpy as np
import pandas as pd
from pathlib import Path

from scripts.data_processing import create_cleaned_df, get_class_label_to_int_mapping
                                    

DATA_DIR = Path('data')
class_label_to_int_mapping = get_class_label_to_int_mapping()

In [10]:
# Read in the sheets we want from the Excel file
sheet_names = ['Seawater - No Heavy Metals', 'Copper', 'Cadmium', 'Lead']
xcel = pd.read_excel(DATA_DIR / 'main.xlsx', sheet_name=sheet_names)

# Create dataframes for each class
seawater = xcel['Seawater - No Heavy Metals']
copper = xcel['Copper']
cadmium = xcel['Cadmium']
lead = xcel['Lead']

In [12]:
create_cleaned_df(copper, 'Cu')

,0,1,2,3,4,5,6,7,8,9,...,993,994,995,996,997,998,999,1000,1001,label
Cu_500_ppb_0,-3.225950,-3.350025,-3.350025,-3.350025,-3.350025,-3.287988,-3.350025,-3.287988,-3.350025,-3.287988,...,8.809325,8.933400,8.995438,9.119513,9.243588,9.367663,9.491738,9.615813,9.739888,1
Cu_500_ppb_1,-4.032438,-4.094475,-4.094475,-4.094475,-4.094475,-4.094475,-4.094475,-4.032438,-4.032438,-4.156513,...,5.955600,6.079675,6.203750,6.265788,6.389863,6.451900,6.700050,6.700050,6.886163,1
Cu_500_ppb_2,-0.496300,-0.558338,-0.558338,-0.496300,-0.558338,-0.496300,-0.558338,-0.434263,-0.558338,-0.496300,...,3.536138,3.536138,3.660213,3.722250,3.846325,3.846325,4.094475,4.156513,4.280588,1
Cu_500_ppb_3,-0.806488,-0.744450,-0.744450,-0.744450,-0.744450,-0.682413,-0.682413,-0.682413,-0.620375,-0.682413,...,3.784288,3.846325,3.970400,4.032438,4.156513,4.280588,4.404663,4.404663,4.590775,1
Cu_500_ppb_4,-0.434263,-0.496300,-0.434263,-0.434263,-0.434263,-0.372225,-0.434263,-0.372225,-0.434263,-0.434263,...,2.791688,2.853725,2.853725,2.977800,3.101875,3.163913,3.225950,3.287988,3.412063,1
Cu_500_ppb_5,-0.558338,-0.558338,-0.558338,-0.558338,-0.496300,-0.496300,-0.496300,-0.496300,-0.558338,-0.496300,...,3.039838,3.039838,3.163913,3.225950,3.350025,3.412063,3.474100,3.598175,3.660213,1
Cu_500_ppb_6,-0.434263,-0.434263,-0.434263,-0.434263,-0.434263,-0.372225,-0.372225,-0.372225,-0.310188,-0.434263,...,2.667613,2.729650,2.791688,2.915763,2.977800,3.039838,3.101875,3.225950,3.350025,1
Cu_500_ppb_7,-0.558338,-0.496300,-0.496300,-0.496300,-0.434263,-0.496300,-0.496300,-0.496300,-0.434263,-0.434263,...,2.977800,3.039838,3.101875,3.163913,3.350025,3.412063,3.474100,3.598175,3.660213,1
Cu_1000_ppb_8,-2.977800,-2.915763,-2.977800,-2.915763,-2.853725,-2.915763,-2.915763,-2.853725,-2.915763,-2.853725,...,5.149113,5.273188,5.459300,5.521338,5.707450,5.831525,6.017638,6.079675,6.327825,1
Cu_1000_ppb_9,-3.225950,-3.101875,-3.163913,-3.039838,-2.977800,-2.915763,-2.915763,-2.915763,-2.915763,-2.853725,...,4.900963,4.963000,5.149113,5.211150,5.397263,5.521338,5.707450,5.893563,6.017638,1


Now all the rows have a uniquely identifiable name. This will help later on when we only want to input certain rows into our model. The columns have been renamed as ints ranging from 0-1001, we don't need to use the actual voltages at any point. Lastly, we've added a 'label' column containing the class label for this analyte as an int (since PyTorch requires your class labels to be ints).

Now let's save the data as csvs in the DATA_DIR.

In [8]:
create_cleaned_df(copper, 'Cu').to_csv(DATA_DIR / 'copper.csv')
create_cleaned_df(cadmium, 'Cd').to_csv(DATA_DIR / 'cadmium.csv')
create_cleaned_df(lead, 'Pb').to_csv(DATA_DIR / 'lead.csv')

We need to slightly modify the code for seawater to remove the 'seawater_' appendix on each row that was added in the `create_cleaned_df` function. I decided not to put any checks in the function to see if the input was seawater or not as this change is only a couple of lines and we are only working with 4 classes. If I had more classes that behaved like seawater in the `create_cleaned_df` function, then I would put the logic to handle this in the function.

In [9]:
seawater = create_cleaned_df(seawater, 'Sw')
# Remove the 'seawater_' from the start of all row names
start_index = len('seawater_')
seawater.index = [name[start_index:] for name in seawater.index]
seawater.to_csv(DATA_DIR / 'seawater.csv')

Now we have the data in a form that is easy to work with and can be fed into our models. Let's do some data exploration.